## Amazon Bedrock Text-to-SQL Evaluation

### Intro and Goal
This Jupyter Notebook is designed to illustrate Text-to-SQL evaluation.

The goal is to take highlight programmatic evaluation metrics as well as LLM as a Judge.

The notebook is structured as follows:
1. Download ground truth dataset comprised of questions and SQL queries for a given database (e.g. Northwind)
2. Evaluate accuracy, cost, and latency of different Text-to-SQL approaches compared to the baseline (zero-shot prompting) for all queries.

In [ ]:
# 1. Import necessary libraries and load environment variables
from dotenv import load_dotenv, find_dotenv
import os
import pandas as pd

# loading environment variables that are stored in local file
local_env_filename = 'dev.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['SQL_DATABASE'] = os.getenv('SQL_DATABASE') # LOCAL, SQLALCHEMY, REDSHIFT
os.environ['SQL_DIALECT'] = os.getenv('SQL_DIALECT') # SQlite, PostgreSQL
os.environ['ENDPOINT_NAME'] = os.getenv('ENDPOINT_NAME')


REGION = os.environ['REGION']
HF_TOKEN = os.environ['HF_TOKEN']
SQL_DATABASE = os.environ['SQL_DATABASE']
SQL_DIALECT = os.environ['SQL_DIALECT']
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']

MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0" #anthropic.claude-3-sonnet-20240229-v1:0" # anthropic.claude-3-haiku-20240307-v1:0 "anthropic.claude-3-5-sonnet-20240620-v1:0" "meta.llama3-1-70b-instruct-v1:0"
EVAL_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

# get ground truth data
file_path = './data/ground_truth.jsonl'
groundtruth_df = pd.read_json(file_path, lines=True)

print(f"Using database: {SQL_DATABASE} with sql dialect: {SQL_DIALECT}")

In [ ]:
# load chromadb
import chromadb
import boto3
from chromadb.config import Settings

# Initialize Chroma client from our persisted store
chroma_client = chromadb.PersistentClient(path="../data/chroma")

from chromadb.utils.embedding_functions import AmazonBedrockEmbeddingFunction
from utils.chroma import BaseRetrievalTask, ChromaDBRetrievalTask
# Define some experiment variables
TITAN_TEXT_EMBED_V2_ID: str = 'amazon.titan-embed-text-v2:0'
COLLECTION_NAME: str = 'sqlsamples_collection'

session = boto3.Session()
embedding_function = AmazonBedrockEmbeddingFunction(
    session=session,
    model_name=TITAN_TEXT_EMBED_V2_ID
)

retrieval_task: BaseRetrievalTask = ChromaDBRetrievalTask(
    chroma_client = chroma_client, 
    collection_name = COLLECTION_NAME,
    embedding_function = embedding_function,
)

### Evaluation

In [ ]:
#2. Grading prompt

evaluation_template = """You are a SQL expert. 
                Your task is to evaluate a given SQL query based on a provided SQL schema and question using the criteria provided below.
 
                Evaluation Criteria (Additive Score, 0-5):
                1. Context: Award 1 point if the generated SQL query uses only information provided in the SQL schema, without introducing external or fabricated details.
                2. Completeness: Add 1 point if the generated SQL query addresses all key elements of the question based on the available SQL schema and Exact Set Match Accuracy (EM) score.
                3. ExecutionAccuracy: Add 1 point if the generated SQL query is very close to the groundtruth answer based on Execution Accuracy score.
                4. Faultless: Add 1 point if the generated SQL query ran without any errors.
                5. ValidEfficiencyScore:  Add 1 point if the runtime of the generated SQL query is similar or better than the the groundtruth qery as measured by the Valid Efficiency Score (VES).
                
                Evaluation Steps:
                1. Read provided context, question and answer carefully.
                2. Go through each evaluation criterion one by one and assess whether the answer meets the criteria.
                3. Compose your reasoning for each critera, explaining why you did or did not award a point. You can only award full points. 
                4. Calculate the total score by summing the points awarded.
                5. Think through the evaluation criteria inside <thinking></thinking> tags. 
                Then, output the total score inside <score></score> tags.
                Review your formatted response. It needs to be valid XML.
    
                Original question:
                <question>
                {question}
                </question>

                SQL schema:
                <sql_schema>
                {sql_schema}
                </sql_schema>

                Generated SQL query based on these instructions:
                <sql_query>
                {sql_query}
                </sql_query>

                SQL result based on the generated SQL query:
                <sql_query_run_result>
                {sql_query_run_result}
                </sql_query_run_result>

                Any SQL errors that might have occured based on the generated SQL query:
                <sql_query_run_error>
                {sql_query_run_error}
                </sql_query_run_error>

                Groundtruth SQL query for comparison with the generated SQL query:
                <groundtruth_sql_query>
                {groundtruth_sql_query}
                </groundtruth_sql_query>
                
                Execution Accuracy, which compares the generated SQL query to the labeled SQL query to determine if its a match or not: 
                <ex_score>
                {ex_score}
                </ex_score>
                
                Exact Set Match Accuracy (EM), which evaluates if the returned result set actually answers the question, regardless of how the query was written: 
                <em_score>
                {em_score}
                </em_score>

                Valid Efficiency Score (VES), which compares the runtime of the SQL provided as groundtruth to the generated SQL query:
                <ves_score>
                {ves_score}
                </ves_score>                
                """

In [ ]:
# 3. Run eval with zero-shot template 
# Zero-shot SQL prompt template to establish baseline
from utils.eval import AnswerTaskRunner

## added "Always prefix table names with the "public." prefix." to support running queries via sqlalchemy 
zero_shot_sql_template = """You are a SQL expert. You will be provided with the original user question and a SQL database schema. 
                Only return the SQL query and nothing else.
                
                User question:
                <user_question>
                {user_question}
                </user_question>

                SQL database schema:
                <sql_database_schema>
                {sql_database_schema}
                </sql_database_schema>
                
                SQL dialect:
                <sql_dialect>
                {sql_dialect}
                </sql_dialect>

                Instructions:
                Generate a SQL query that answers the original user question.
                Use the schema, first create a syntactically correct {sql_dialect} query to answer the question. 
                Never query for all the columns from a specific table, only ask for a few relevant columns given the question.
                Always prefix table names with the "public." prefix.
                Pay attention to use only the column names that you can see in the schema description. 
                Be careful to not query for columns that do not exist. 
                Pay attention to which column is in which table. 
                Also, qualify column names with the table name when needed.
                If you cannot answer the user question with the help of the provided SQL database schema, 
                then output that this question question cannot be answered based of the information stored in the database.
                You are required to use the following format, each taking one line:
                Return the sql query inside the <sql></sql> tab.
                """



answer_results1: pd.DataFrame = AnswerTaskRunner(groundtruth_df[:10],
                 model_id=MODEL_ID,
                 eval_model_id=EVAL_MODEL_ID,
                 sql_database=SQL_DATABASE,
                 sql_dialect=SQL_DIALECT,
                 prompt_template=zero_shot_sql_template,
                 prompt_eval_template=evaluation_template).run()

answer_results1.to_json('./data/zero-shot-graded.jsonl', orient='records', lines=True, force_ascii=False, date_format='iso', default_handler=str)

In [ ]:
answer_results1.head(5)

In [ ]:
# 4. Run eval with few-shot template

few_shot_sql_template = """You are a SQL expert. You will be provided with the original user question and a SQL database schema. 
                Only return the SQL query and nothing else.
                Here is the original user question.
                <user_question>
                {user_question}
                </user_question>

                Here is the SQL database schema.
                <sql_database_schema>
                {sql_database_schema}
                </sql_database_schema>

                Here are some examples of SQL queries that answer similar questions:
                <sql_examples>
                {sql_examples}
                </sql_examples>
                
                Instructions:
                Generate a SQL query that answers the original user question.
                Use the schema, first create a syntactically correct {sql_dialect} query to answer the question. 
                Never query for all the columns from a specific table, only ask for a few relevant columns given the question.
                Always prefix table names with the "public." prefix.
                Pay attention to use only the column names that you can see in the schema description. 
                Be careful to not query for columns that do not exist. 
                Pay attention to which column is in which table. 
                Also, qualify column names with the table name when needed.
                If you cannot answer the user question with the help of the provided SQL database schema, 
                then output that this question question cannot be answered based of the information stored in the database.
                You are required to use the following format, each taking one line.
                Return the sql query inside the <SQL></SQL> tab.
                """
answer_results2: pd.DataFrame = AnswerTaskRunner(groundtruth_df,
                 model_id=MODEL_ID,
                 eval_model_id=EVAL_MODEL_ID,
                 sql_database=SQL_DATABASE,
                 sql_dialect=SQL_DIALECT,
                 prompt_template=few_shot_sql_template,
                 prompt_eval_template=evaluation_template
                 retrieval_task=retrieval_task).run()

answer_results2.to_json('./data/few-shot-graded.jsonl', orient='records', lines=True, force_ascii=False, date_format='iso', default_handler=str)

In [ ]:
# 5. Compare results
from utils.util import Util
util = Util()
util.compare_results(answer_results2, answer_results1)
print('zero-shot score distribution')
util.visualize_distribution(answer_results1, key='score')
print('few-shot score distribution')
util.visualize_distribution(answer_results2,key='score')

In [ ]:
# 6. Review results
print(f'sum of zero_shot_cost: {answer_results1["cost"].sum()}')
print(f'sum of few_shot_cost: {answer_results2["cost"].sum()}')
print(f'avg of zero_shot_query_time: {answer_results1["latency"].mean()}')
print(f'avg of few_shot_query_time: {answer_results2["latency"].mean()}')

# Execution Accuracy, which compares the generated SQL query to the ground truth SQL query to determine if its a match
print(f'avg of ex_score of zero_shot: {answer_results1["ex_score"].mean()}')
print(f'avg of ex_score of few_shot: {answer_results2["ex_score"].mean()}')
# Exact Set Match Accuracy (EM), which evaluates if the generated SQL query resultset matches the ground truth resultset
print(f'avg of em_score of zero_shot: {answer_results1["em_score"].mean()}')
print(f'avg of em_score of few_shot: {answer_results2["em_score"].mean()}')
# Valid Efficiency Score (VES), which compares the  generated router SQL query runtime provided to the ground truth SQL query runtime
print(f'avg of ves_score of zero_shot: {answer_results1["ves_score"].mean()}')
print(f'avg of ves_score of few_shot: {answer_results2["ves_score"].mean()}')

In [ ]:
# 7. Run eval with finetuned LLM and zero-shot template
from utils.eval import AnswerTaskRunner

finetuned_sql_template = """{user_question} \n\n### Context {sql_database_schema}"""

answer_results3: pd.DataFrame = AnswerTaskRunner(groundtruth_df[:10],
                 model_id="",
                 endpoint_name=ENDPOINT_NAME,
                 max_token_count=512,
                 eval_model_id=EVAL_MODEL_ID,
                 sql_database=SQL_DATABASE,
                 sql_dialect=SQL_DIALECT,
                 prompt_template=finetuned_sql_template,
                 prompt_eval_template=evaluation_template,
                 region=REGION).run()

answer_results3.to_json('./data/zero-shot-finetuned-graded.jsonl', orient='records', lines=True, force_ascii=False, date_format='iso', default_handler=str)

In [ ]:
answer_results3.head(5)

In [ ]:
# 11. Compare fine-tuned model results with zero-shot prompting

print(f'avg of zero-shot_query_time: {answer_results1["latency"].mean()}')
print(f'avg of finetuned_query_time: {answer_results3["latency"].mean()}')

# Execution Accuracy, which compares the generated SQL query to the ground truth SQL query to determine if its a match
print(f'avg of ex_score of zero_shot: {answer_results1["ex_score"].mean()}')
print(f'avg of ex_score of finetuned: {answer_results3["ex_score"].mean()}')
# Exact Set Match Accuracy (EM), which evaluates if the generated SQL query resultset matches the ground truth resultset
print(f'avg of em_score of zero_shot: {answer_results1["em_score"].mean()}')
print(f'avg of em_score of finetuned: {answer_results3["em_score"].mean()}')
# Valid Efficiency Score (VES), which compares the  generated router SQL query runtime provided to the ground truth SQL query runtime
print(f'avg of ves_score of zero_shot: {answer_results1["ves_score"].mean()}')
print(f'avg of ves_score of finetuned: {answer_results3["ves_score"].mean()}')

In [ ]:
util = Util()
metrics = ['score', 'ex_score', 'em_score', 'ves_score'] # 'latency' ,'cost'

print('zero-shot score distribution')
util.visualize_distribution(answer_results1, key='score')
print('finetuned score distribution')
util.visualize_distribution(answer_results3, key='score')
util.compare_results(answer_results1, answer_results3, metrics)

### Conclusion
XXX